In [64]:
#데이터 시각화 보여주기(가장 많이 떠든 사람)

import re
from konlpy.tag import Kkma
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from collections import Counter
from soynlp.normalizer import *
text=""
with open(r"C:\Users\Msi\Desktop\jongmanhoney.txt",encoding="utf-8")as f:
    text=f.read()
r=re.compile("[#]")
list_data = re.findall('\d+',text)
list_data=[w for w in list_data if len(w)>3 and len(w)<5 and w!='2021'] #강제로 처리해주기
list_data=sorted(list_data)
bow={}
for w in list_data:
    if w not in bow.keys():
        bow[w]=0
    else:
        bow[w]+=1
        
bow={w:bow[w] for w in bow.keys() if bow[w]>100}
print(bow)

from matplotlib import font_manager

font_fname = 'C:\\Users\\Msi\\Desktop\\BlackHanSans-Regular.ttf'

font_family = font_manager.FontProperties(fname=font_fname).get_name()

plt.rcParams["font.family"] = font_family
wc=WordCloud(font_fname,width=1200,height=800,scale=2.0,max_font_size=250)
gen=wc.generate_from_frequencies(bow)
plt.figure()
plt.imshow(gen,interpolation='bilinear')
wc.to_file("발표.png")
plt.close()

{'0146': 226, '0251': 108, '3671': 256, '4613': 3550, '4849': 2876, '5946': 2065, '6377': 3599, '6573': 650, '7720': 1223}


In [3]:
#사전 훈련된 Word2Vec 임베딩(Pre-trained Word2Vec embedding)
#사전에 학습시켜놓은 임베딩벡터들을 가지고 와서 모델의 입력으로 사용하는 것이 더 좋은 성능

#영어
import gensim
model=gensim.models.KeyedVectors.load_word2vec_format(r'C:\Users\Msi\Desktop\GoogleNews-vectors-negative300.bin.gz',binary=True)
print(model.vectors.shape)
#모델의 크기는 3000000 x 300 3000000만개의 단어와 각 단어의 차원은 300


(3000000, 300)


In [8]:
print(model.similarity('this','is'))#두 단어의 유사도 계산하기
print(model.similarity('marvel','avengers'))

0.40797037
0.11652229


In [16]:
#한국어 Word2Vec
import gensim
model=gensim.models.Word2Vec.load(r'C:\Users\Msi\Desktop\ko.bin')
result=model.wv.most_similar("광해")
print(result)

[('광무', 0.5746680498123169), ('건양', 0.5569872856140137), ('중평', 0.5530575513839722), ('융희', 0.5521409511566162), ('초평', 0.5479120016098022), ('충혜왕', 0.5460560321807861), ('효공왕', 0.5406419038772583), ('무덕', 0.5385192632675171), ('백낙', 0.5381578803062439), ('충정왕', 0.5341129302978516)]


In [32]:
#20뉴스그룸 데이터 전처리하기
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn.datasets import fetch_20newsgroups
from tensorflow.keras.preprocessing.text import Tokenizer

dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents=dataset.data
print("총 샘플 수 : ",len(documents))
documents[0]

총 샘플 수 :  11314


"Well i'm not sure about the story nad it did seem biased. What\nI disagree with is your statement that the U.S. Media is out to\nruin Israels reputation. That is rediculous. The U.S. media is\nthe most pro-israeli media in the world. Having lived in Europe\nI realize that incidences such as the one described in the\nletter have occured. The U.S. media as a whole seem to try to\nignore them. The U.S. is subsidizing Israels existance and the\nEuropeans are not (at least not to the same degree). So I think\nthat might be a reason they report more clearly on the\natrocities.\n\tWhat is a shame is that in Austria, daily reports of\nthe inhuman acts commited by Israeli soldiers and the blessing\nreceived from the Government makes some of the Holocaust guilt\ngo away. After all, look how the Jews are treating other races\nwhen they got power. It is unfortunate.\n"

In [21]:
news_df=pd.DataFrame({"document":documents})
news_df['clean_doc']=news_df['document'].str.replace("[^a-zA-Z]"," ")
news_df['clean_doc']=news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
news_df['clean_doc']=news_df['clean_doc'].apply(lambda x: x.lower())#소문자 변환

In [22]:
news_df.isnull().values.any()

False

In [23]:
news_df.replace("",float("NaN"),inplace=True)
news_df.isnull().values.any() #Null값이 있음을 확인

True

In [24]:
news_df.dropna(inplace=True)
print('총 샘플 수 : ',len(news_df)) #샘플수를 줄였다!

총 샘플 수 :  10995


In [25]:
stop_words=stopwords.words('english')#불용어
tokenized_doc=news_df['clean_doc'].apply(lambda x : x.split())
tokenized_doc=tokenized_doc.apply(lambda x : [item for item in x if item not in stop_words])
tokenized_doc=tokenized_doc.to_list()
#불용어를 제거해서 단어의 수를 줄였다

In [26]:
#단어 1개 이하인 샘플의 인덱스를 찾아서 저장하고, 해당 샘플들은 제거.
drop_train=[index for index, sentence in enumerate(tokenized_doc) if len(sentence)<=1]
tokenized_doc=np.delete(tokenized_doc,drop_train,axis=0)
print('총 샘플수 : ',len(tokenized_doc))

총 샘플수 :  10940


C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [33]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(tokenized_doc)

word2idx=tokenizer.word_index
idx2word={v:k for k,v in word2idx.items()}
encoded=tokenizer.texts_to_sequences(tokenized_doc)

In [28]:
print(encoded[:2])

[[9, 59, 603, 207, 3278, 1495, 474, 702, 9470, 13686, 5533, 15227, 702, 442, 702, 70, 1148, 1095, 1036, 20294, 984, 705, 4294, 702, 217, 207, 1979, 15228, 13686, 4865, 4520, 87, 1530, 6, 52, 149, 581, 661, 4406, 4988, 4866, 1920, 755, 10668, 1102, 7837, 442, 957, 10669, 634, 51, 228, 2669, 4989, 178, 66, 222, 4521, 6066, 68, 4295], [1026, 532, 2, 60, 98, 582, 107, 800, 23, 79, 4522, 333, 7838, 864, 421, 3825, 458, 6488, 458, 2700, 4730, 333, 23, 9, 4731, 7262, 186, 310, 146, 170, 642, 1260, 107, 33568, 13, 985, 33569, 33570, 9471, 11491]]


In [29]:
vocab_size=len(word2idx)+1
print("단어 집합의 크기: ",vocab_size)

단어 집합의 크기:  64277


In [35]:
#네거티브 샘플링을 통한 데이터셋 구성하기
from tensorflow.keras.preprocessing.sequence import skipgrams
#네거티브 샘플링
skip_grams=[skipgrams(sample,vocabulary_size=vocab_size,window_size=10) for sample in encoded[:10]]
pairs,labels=skip_grams[0][0],skip_grams[0][1]
for i in range(5):
    print("({:s} ({:d}),{:s} ({:d}))->{:d}".format(idx2word[pairs[i][0]],pairs[i][0],
                                                  idx2word[pairs[i][1]],pairs[i][1],
                                                  labels[i]))

(government (51),annoying (3107))->0
(received (634),interlaced (6718))->0
(shame (4988),received (634))->1
(world (70),occured (4294))->1
(subsidizing (15228),babes (11210))->0


In [36]:
print('전체 샘플 수 :',len(skip_grams))

전체 샘플 수 : 10


In [37]:
print(len(pairs))
print(len(labels))

2220
2220


In [41]:
#Skip-Gram with Negative Sampling(SGNS) 구현하기
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding,Reshape,Activation,Input
from tensorflow.keras.layers import Dot
from tensorflow.keras.utils import plot_model
from IPython.display import SVG

#임베딩 차원은 100 사용자가 정하는 하이퍼파라미터다.
embed_size=100
w_inputs=Input(shape=(1,),dtype='int32')
word_embedding=Embedding(vocab_size,embed_size)(w_inputs)

#주변 단어를 위한 임베딩 테이블
c_inputs=Input(shape=(1, ),dtype='int32')
context_embedding=Embedding(vocab_size,embed_size)(c_inputs)

dot_product=Dot(axes=2)([word_embedding,context_embedding])
dot_product=Reshape((1,),input_shape=(1,1))(dot_product)
output=Activation('sigmoid')(dot_product)
model=Model(inputs=[w_inputs,c_inputs],outputs=output)
model.summary()
model.compile(loss='binary_crossentropy',optimizer='adam')
plot_model(model,to_file='model3.png',show_shapes=True,show_layer_names=True,rankdir='TB')

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_7 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 1, 100)       6427700     input_6[0][0]                    
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, 1, 100)       6427700     input_7[0][0]                    
______________________________________________________________________________________________

In [45]:
#모델의 학습은 5에포크 수행하겠다.
for epoch in range(1, 6):
    loss = 0
    for _, elem in enumerate(skip_grams):
        first_elem = np.array(list(zip(*elem[0]))[0], dtype='int32')
        second_elem = np.array(list(zip(*elem[0]))[1], dtype='int32')
        labels = np.array(elem[1], dtype='int32')
        X = [first_elem, second_elem]
        Y = labels
        loss += model.train_on_batch(X,Y)  
    print('Epoch :',epoch, 'Loss :',loss)

Epoch : 1 Loss : 6.766035974025726
Epoch : 2 Loss : 6.707617878913879
Epoch : 3 Loss : 6.636330723762512
Epoch : 4 Loss : 6.550922453403473
Epoch : 5 Loss : 6.450494885444641


In [43]:
import gensim

In [46]:
f=open('vectors.txt','w')
f.write('{} {}\n'.format(vocab_size-1,embed_size))
vectors=model.get_weights()[0]
for word, i in tokenizer.word_index.items():
    f.write('{} {}\n'.format(word, ' '.join(map(str,list(vectors[i,:])))))
f.close()

w2v=gensim.models.KeyedVectors.load_word2vec_format('./vectors.txt',binary=False)
w2v.most_similar(positive=['soldiers'])

[('commited', 0.41037195920944214),
 ('buddist', 0.3853633403778076),
 ('jails', 0.3764612674713135),
 ('binah', 0.3735484778881073),
 ('onatal', 0.3721325397491455),
 ('oooooo', 0.36630985140800476),
 ('falls', 0.3662239909172058),
 ('undercooking', 0.36434799432754517),
 ('extermination', 0.3570270240306854),
 ('benefitting', 0.3548571467399597)]